<a href="https://colab.research.google.com/github/diegooyarcet/headline-analysis/blob/main/headline_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Headline Analysis

## LA TERCERA
### Objetivo: Analizar el lenguaje utilizado en los titulares del diario La Tercera, específicamente aquellos que se encuentran en la sección "Lo Último", enfocándose en identificar patrones, temas frecuentes y características lingüísticas.

### Herramientas:
* Python
* NLTK
* BeautifulSoup (bs4)
* requests
* matplotlib
* re
* pandas

In [1]:
!pip install requests beautifulsoup4 pandas

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = 'https://www.latercera.com/lo-ultimo'

In [4]:
df = pd.DataFrame(columns=['tag', 'headline'])

In [5]:
try:
  response = requests.get(url)
except Exception as exc:
  print(f"Error: {exc}")

In [6]:
if response.status_code == 200:
  print("Solicitud exitosa")
  html_content = response.text
else:
  print(f"Error en la solicitud. Código de estado: {response.status_code}")

Solicitud exitosa


In [7]:
lo_ultimo = BeautifulSoup(html_content, 'html.parser')
paginator = lo_ultimo.find('div', class_='pagination')
nav_paginator = paginator.find('nav')
ul_paginator = nav_paginator.find('ul')
pagination_info = ul_paginator.find('li', class_='pagination-info')
text_pagination_info = pagination_info.get_text(strip=True)
match = re.search(r"de\s+(\d+)", text_pagination_info)

if match:
  total_paginas = int(match.group(1))
  print(f"Número total de páginas: {total_paginas}")
else:
  print("No se encontró el patrón de número de páginas en el texto.")

Número total de páginas: 667


In [11]:
df = df[0:0]

In [12]:
total_paginas = 2
for pagina in range(1, total_paginas + 1):
  print(f"PÁGINA {pagina}")
  url_pagina = f"{url}/page/{pagina}"
  response = requests.get(url_pagina)

  if response.status_code == 200:
    html_lo_ultimo = response.text
    soup_lo_ultimo = BeautifulSoup(html_lo_ultimo, 'html.parser')

    if soup_lo_ultimo:
      section_lo_ultimo = soup_lo_ultimo.find('section', class_='top-mainy')

      if section_lo_ultimo:
        div_headlines = section_lo_ultimo.find_all('div', class_='headline | width_full hl')

        if len(div_headlines) > 0:
          for headline in div_headlines:
            tag = ''
            titular = ''
            h3_headline = headline.find('h3')

            if h3_headline:
              anchor_headline = h3_headline.find('a')

              if anchor_headline:
                tag_element = anchor_headline.find('span', class_='tag')

                if tag_element:
                  tag = tag_element.get_text(strip=True)
                else:
                  tag = ''
                  print("No hay tag")

                titular = anchor_headline.get_text(strip=True)

                if titular:
                  titular = titular.replace(tag, '', 1).strip()
                else:
                  titular = ''
                  print("No hay titular")

                df.loc[len(df)] = [tag, titular]
              else:
                print("No se encontró el h3")
            else:
              print("No se encontró el anchor")
        else:
          print(f"No se encontraron titulares en la página {pagina}")
      else:
        print(f"No se encontró el section de títulares de la página {pagina}")
    else:
      print(f"No se encontro el soup de la página {pagina}")
  else:
    print(f"No se pudo acceder a la página: {url_pagina}")

print(df)

PÁGINA 1
PÁGINA 2
                        tag                                           headline
0                 Elon Musk  “Nazi, las pelotas”: la férrea defensa de Javi...
1                      La U  Con show de drones incluido: la U presenta su ...
2                 Marejadas  Emiten nuevo aviso de marejadas para las costa...
3                    EE.UU.  Trump advierte más sanciones a Rusia si Putin ...
4                 Venezuela  Familiares de presos políticos en Venezuela pi...
5      Ley de Transparencia  Instruyen investigaciones sumarias en tres sub...
6                    Fútbol  “No dan los plazos para que se juegue”: Sifup ...
7                    EE.UU.  Trump autoriza a agentes federales de inmigrac...
8          Hablemos de amor  Mis éxitos profesionales se empezaron a transf...
9      Incendios forestales  Incendios forestales: Chile registra 17 olas d...
10                   Fútbol  Colo Colo define su último fichaje: Tomás Alar...
11  Correos de los Lectores  Inter